# MNIST 

Página 130 del Aurelien Geron 


En este capítulo utilizaremos el conjunto de datos MNIST, que es un conjunto de 70.000 pequeñas imágenes de dígitos escritas a mano por estudiantes de secundaria y empleados de la Oficina del Censo de EE.UU.

Cada imagen está etiquetada con el dígito que representa.

Este conjunto se ha estudiado tanto que a menudo se lo llama el "hola mundo" del aprendizaje automático: cada vez que las personas crean un nuevo algoritmo de clasificación, sienten curiosidad por ver cómo funcionará en MNIST, y cualquiera que aprenda el AA lo abordará tarde o temprano.

Scikit-Learn proporciona muchas funciones de ayuda para descargar conjuntos de datos populares.

MNIST es uno de ellos. El siguiente código obtiene el conjunto de datos MNIST:

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

In [2]:
mnist.keys() 
# aplicación del método keys() para obtener lista de atributos

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

Los conjuntos de datos cargados por Scikit-Learn generalmente tienen una estructura de diccionario similar, que incluye lo siguiente:

* Una clave `DESCR` que describe el conjunto de datos  
* Una clave `data` que contiene una matriz con una fila por instancia y una columna por   
  característica. 
* Una clave de `target` que contiene una matriz con las etiquetas

In [3]:
type(mnist) 

sklearn.utils.Bunch

In [4]:
type(mnist.data)  

pandas.core.frame.DataFrame

In [7]:
mnist.data.head()   

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
mnist.target 

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: category
Categories (10, object): [0, 1, 2, 3, ..., 6, 7, 8, 9]

Veamos estas matrices:

In [9]:
X, y = mnist["data"], mnist["target"] # aplicando asignación múltiple 

In [10]:
type(X) 

pandas.core.frame.DataFrame

In [11]:
y.shape


(70000,)

In [12]:
y 

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: category
Categories (10, object): [0, 1, 2, 3, ..., 6, 7, 8, 9]

In [13]:
type(y) 

pandas.core.series.Series

In [14]:
type(X)

pandas.core.frame.DataFrame

Hay 70.000 imágenes y cada imagen tiene 784 características.

Esto se debe a que cada imagen tiene $28 \times 28$ píxeles y cada característica simplemente representa la intensidad de un píxel, de 0 (blanco) a 255 (negro).

Echemos un vistazo a un dígito del conjunto de datos.

Todo lo que necesita hacer es tomar el vector de características de una instancia, remodelarlo a una matriz $28 \times 28$ y mostrarlo usando la función `imshow()` de Matplotlib:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

In [ ]:
some_digit = X.iloc[0,:]  

In [ ]:
some_digit 

In [ ]:
some_digit.values


In [ ]:
some_digit_image = some_digit.values.reshape(28, 28)

In [ ]:
plt.imshow(some_digit_image,cmap='binary') 
plt.axis("off")
plt.show()

Esto parece un 5 y, de hecho, eso es lo que nos dice la etiqueta:

In [ ]:
y[0]

Tenga en cuenta que la etiqueta es una cadena. 

La mayoría de los algoritmos de AA esperan números, por lo que conviene convertir `y[0]` en un número entero:

In [ ]:
y = y.astype(np.int) 
y[0]  

Para darle una idea de la complejidad de la tarea de clasificación, la Figura 3-1 muestra algunas imágenes más del conjunto de datos MNIST.

In [ ]:
from IPython.display import Image
Image('figure_3_1.jpg',width=400,height=200) 

## ¡Pero espera!

Siempre debe crear un conjunto de prueba y dejarlo a un lado antes de inspeccionar los datos de cerca.

* Podemos dividir el conjunto de datos MNIST en un conjunto de entrenamiento   
  (las primeras 60,000 imágenes) y   
* un conjunto de prueba (las últimas 10,000 imágenes):

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

El conjunto de entrenamiento ya está barajado para nosotros, lo cual es bueno porque esto garantiza que todos los pliegues de validación cruzada serán similares. 

Además, algunos algoritmos de aprendizaje son sensibles al orden de las instancias de capacitación y funcionan mal si obtienen muchas instancias similares seguidas.

Mezclar el conjunto de datos garantiza que esto no suceda.

## Entrenamiento de un clasificador binario

Let’s simplify the problem for now and only try to identify one digit—for example, the number 5. 

This “5-detector” will be an example of a binary
classifier, capable of distinguishing between just two classes, 5 and not-5.
Let’s create the target vectors for this classification task:

In [ ]:
y_train_5 = (y_train == 5) # True for all 5s, False for all other digits
y_test_5 = (y_test == 5)

Ahora elija un clasificador y entrenelo.   

Un buen lugar para comenzar es con un clasificador de gradiente descendente estocástico (SGD), utilizando la clase `SGDClassifier` de Scikit-Learn.

Este clasificador tiene la ventaja de poder manejar conjuntos de datos muy grandes de manera eficiente.

Vamos a crear un SGDClassifier y entrenarlo en todo el conjunto de entrenamiento:

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

### TIP

El SGDClassifier se basa en la aleatoriedad durante el entrenamiento (de ahí el nombre "estocástico").

Si desea resultados reproducibles, debe establecer el parámetro `random_state`.

Ahora podemos usarlo para detectar imágenes del número 5:

In [ ]:
sgd_clf.predict([some_digit])